In [143]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from collections import Counter
from PIL import Image

folder_path = 'aritzia_images'
images = []
types = []
colors = []

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        name_part = filename.rsplit('.', 1)[0]
        type_and_color = name_part.split('_', 1)
        
        item_type = type_and_color[0].split()[-1].strip()
        color = type_and_color[1].split()[-1].strip()
        
        if color.lower() != "unknown":
            types.append(item_type)
            colors.append(color)
           
            img = Image.open(os.path.join(folder_path, filename)).resize((128, 128))
            images.append(np.array(img) / 255.0)

images = np.array(images)

types_array = np.array(types)
colors_array = np.array(colors)

num_types = len(np.unique(types_array))
num_colors = len(np.unique(colors_array))
print(f"Number of unique types: {num_types}")
print(f"Number of unique colors: {num_colors}")

type_encoder = pd.factorize(pd.Series(types_array))
color_encoder = pd.factorize(pd.Series(colors_array))

type_labels = tf.keras.utils.to_categorical(type_encoder[0], num_classes=num_types)
color_labels = tf.keras.utils.to_categorical(color_encoder[0], num_classes=num_colors)

X_train, X_val, y_type_train, y_type_val, y_color_train, y_color_val = train_test_split(
    images, type_labels, color_labels, test_size=0.2, random_state=42
)

print("\nData shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_type_train: {y_type_train.shape}")
print(f"y_type_val: {y_type_val.shape}")
print(f"y_color_train: {y_color_train.shape}")
print(f"y_color_val: {y_color_val.shape}")

input_layer = tf.keras.layers.Input(shape=(128, 128, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
shared_features = tf.keras.layers.Flatten()(x)

type_x = tf.keras.layers.Dense(512, activation='relu')(shared_features)
type_x = tf.keras.layers.Dropout(0.5)(type_x)
type_x = tf.keras.layers.Dense(256, activation='relu')(type_x)
type_output = tf.keras.layers.Dense(num_types, activation='softmax', name='type_output')(type_x)

color_x = tf.keras.layers.Dense(512, activation='relu')(shared_features)
color_x = tf.keras.layers.Dropout(0.5)(color_x)
color_x = tf.keras.layers.Dense(256, activation='relu')(color_x)
color_output = tf.keras.layers.Dense(num_colors, activation='softmax', name='color_output')(color_x)

model = tf.keras.models.Model(
    inputs=input_layer,
    outputs={'type_output': type_output, 'color_output': color_output}
)

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'type_output': 'categorical_crossentropy',
        'color_output': 'categorical_crossentropy'
    },
    metrics={
        'type_output': 'accuracy',
        'color_output': 'accuracy'
    }
)

history = model.fit(
    x=X_train,
    y={
        'type_output': y_type_train,
        'color_output': y_color_train
    },
    validation_data=(
        X_val,
        {
            'type_output': y_type_val,
            'color_output': y_color_val
        }
    ),
    epochs=20,
    batch_size=32,
)

results = model.evaluate(
    X_val,
    {
        'type_output': y_type_val,
        'color_output': y_color_val
    }
)

print("\nFinal Results:")
metrics = dict(zip(model.metrics_names, results))
print("\nAll metrics:", metrics)

Number of unique types: 50
Number of unique colors: 207

Data shapes:
X_train: (1676, 128, 128, 3)
X_val: (419, 128, 128, 3)
y_type_train: (1676, 50)
y_type_val: (419, 50)
y_color_train: (1676, 207)
y_color_val: (419, 207)


Model: "functional_42"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_42      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_126 (Conv2D) │ (None, 126, 126,  │        896 │ input_layer_42[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_126   │ (None, 63, 63,    │          0 │ conv2d_126[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_127 (Conv2D) │ (None, 61, 61,    │     18,496 │ max_pooling2d_12… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_127   │ (None, 30, 30,    │          0 │ conv2d_127[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_128 (Conv2D) │ (None, 28, 28,    │     73,856 │ max_pooling2d_12… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_128   │ (None, 14, 14,    │          0 │ conv2d_128[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_38          │ (None, 25088)     │          0 │ max_pooling2d_12… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_61 (Dense)    │ (None, 512)       │ 12,845,568 │ flatten_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_59 (Dense)    │ (None, 512)       │ 12,845,568 │ flatten_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_54          │ (None, 512)       │          0 │ dense_61[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_53          │ (None, 512)       │          0 │ dense_59[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_62 (Dense)    │ (None, 256)       │    131,328 │ dropout_54[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_60 (Dense)    │ (None, 256)       │    131,328 │ dropout_53[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_output        │ (None, 207)       │     53,199 │ dense_62[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ type_output (Dense) │ (None, 50)        │     12,850 │ dense_60[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 26,113,089 (99.61 MB)

 Trainable params: 26,113,089 (99.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 150ms/step - color_output_accuracy: 0.2205 - color_output_loss: 4.1745 - loss: 7.5442 - type_output_accuracy: 0.1483 - type_output_loss: 3.3693 - val_color_output_accuracy: 0.2291 - val_color_output_loss: 3.5367 - val_loss: 6.1749 - val_type_output_accuracy: 0.3437 - val_type_output_loss: 2.5920
Epoch 2/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 8s 145ms/step - color_output_accuracy: 0.2909 - color_output_loss: 3.1281 - loss: 5.5066 - type_output_accuracy: 0.3845 - type_output_loss: 2.3781 - val_color_output_accuracy: 0.2625 - val_color_output_loss: 3.1453 - val_loss: 5.1989 - val_type_output_accuracy: 0.5012 - val_type_output_loss: 2.0153
Epoch 3/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 8s 147ms/step - color_output_accuracy: 0.3157 - color_output_loss: 2.9167 - loss: 4.8121 - type_output_accuracy: 0.4836 - type_output_loss: 1.8953 - val_color_output_accuracy: 0.3198 - val_color_output_loss: 3.0758 - val_loss: 4.9616 - val_type_output_accuracy: 0.5155 - val_type_output

In [145]:
model.save('clothing_classifier.keras')

In [147]:
def predict_image(model, image_path, type_encoder, color_encoder):
    try:
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
        
        img = Image.open(image_path)
        
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGB', img.size, (255, 255, 255))
            if img.mode == 'RGBA':
                background.paste(img, mask=img.split()[3])
            else:
                background.paste(img)
            img = background
        elif img.mode != 'RGB':
            img = img.convert('RGB')
            
        img = img.resize((128, 128))
        
        img_array = np.array(img) / 255.0
        
        if img_array.shape != (128, 128, 3):
            print(f"Unexpected image shape: {img_array.shape}")
            raise ValueError(f"Expected image shape (128, 128, 3), got {img_array.shape}")
            
        img_array = np.expand_dims(img_array, axis=0)
        
        predictions = model.predict(img_array)
        
        type_pred_idx = np.argmax(predictions['type_output'][0])
        color_pred_idx = np.argmax(predictions['color_output'][0])
        
        type_confidence = predictions['type_output'][0][type_pred_idx]
        color_confidence = predictions['color_output'][0][color_pred_idx]
        
        predicted_type = type_encoder[1][type_pred_idx]
        predicted_color = color_encoder[1][color_pred_idx]
        
        return predicted_type, predicted_color, type_confidence, color_confidence
        
    except Exception as e:
        print(f"Error processing image: {e}")
        print(f"Image mode: {img.mode if 'img' in locals() else 'unknown'}")
        print(f"Image size: {img.size if 'img' in locals() else 'unknown'}")
        return None

def process_and_predict(image_path, model, type_encoder, color_encoder):
    print(f"Processing image: {image_path}")
    result = predict_image(model, image_path, type_encoder, color_encoder)
    
    if result is not None:
        pred_type, pred_color, type_conf, color_conf = result
        print(f"Predicted Type: {pred_type} (Confidence: {type_conf:.2f})")
        print(f"Predicted Color: {pred_color} (Confidence: {color_conf:.2f})")
    else:
        print("Could not make prediction due to error processing the image.")

In [149]:
loaded_model = tf.keras.models.load_model('clothing_classifier.keras')

In [151]:
image_path = 'green_dress.jpg'
pred_type, pred_color, type_conf, color_conf = predict_image(
    loaded_model, 
    image_path, 
    type_encoder, 
    color_encoder
)

print(f"Predicted Type: {pred_type} (Confidence: {type_conf:.2f})")
print(f"Predicted Color: {pred_color} (Confidence: {color_conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Type: DRESS (Confidence: 1.00)
Predicted Color: OLIVE (Confidence: 0.93)


In [153]:
image_path = 'red_dress.jpg'
pred_type, pred_color, type_conf, color_conf = predict_image(
    loaded_model, 
    image_path, 
    type_encoder, 
    color_encoder
)

print(f"Predicted Type: {pred_type} (Confidence: {type_conf:.2f})")
print(f"Predicted Color: {pred_color} (Confidence: {color_conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Type: DRESS (Confidence: 1.00)
Predicted Color: RED (Confidence: 1.00)


In [159]:
image_path = 'grey_sweater.jpg'
pred_type, pred_color, type_conf, color_conf = predict_image(
    loaded_model, 
    image_path, 
    type_encoder, 
    color_encoder
)

print(f"Predicted Type: {pred_type} (Confidence: {type_conf:.2f})")
print(f"Predicted Color: {pred_color} (Confidence: {color_conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Type: BOMBER (Confidence: 0.62)
Predicted Color: CHARCOAL (Confidence: 1.00)


In [161]:
image_path = 'red_dress_2.jpg'
pred_type, pred_color, type_conf, color_conf = predict_image(
    loaded_model, 
    image_path, 
    type_encoder, 
    color_encoder
)

print(f"Predicted Type: {pred_type} (Confidence: {type_conf:.2f})")
print(f"Predicted Color: {pred_color} (Confidence: {color_conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Type: DRESS (Confidence: 1.00)
Predicted Color: RED (Confidence: 1.00)
